# Data Cleaning Listado de Listado de Beneficiarios 2020

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_20 -> Dataset con el listado de TODOS los beneficiarios de 2020.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2020

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_20 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2020.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_20 = beneficiarios_20[~((beneficiarios_20['ENTIDAD'] == 'NACIONAL') & (beneficiarios_20['MUNICIPIO'] == 'NACIONAL') & (beneficiarios_20['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_20.shape

(369040, 16)

In [6]:
beneficiarios_20['ENTIDAD'].unique()

array(['GUERRERO', 'MORELOS', 'TLAXCALA', 'PUEBLA', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [7]:
beneficiarios_20.dropna(inplace=True)

In [8]:
# Obtenemos las localidades únicas en el dataset.
Localidades_20 = beneficiarios_20[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_20 = Localidades_20.drop_duplicates()


In [9]:
Localidades_20['ENTIDAD_c_benef'] = Localidades_20['ENTIDAD'].apply(clean_text)
Localidades_20['MUNICIPIO_c_benef'] = Localidades_20['MUNICIPIO'].apply(clean_text)
Localidades_20['LOCALIDAD_c_benef'] = Localidades_20['LOCALIDAD'].apply(clean_text)

In [10]:
# Creamos las dos keys de beneficiarios 2019
Localidades_20['KEY_benef_mun'] = Localidades_20['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_20[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_20['KEY_benef_loc'] = Localidades_20['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_20[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_20['LOCALIDAD_c_benef'].astype(str)

In [11]:
Localidades_20.shape

(5381, 8)

In [12]:
# Obtenemos las localidades únicas en el dataset.
Municipios_20 = beneficiarios_20[['ENTIDAD', 'MUNICIPIO']]
Municipios_20 = Municipios_20.drop_duplicates()

In [13]:
# Estandarizamos la limpieza de los datos
Municipios_20['ENTIDAD_c_benef'] = Municipios_20['ENTIDAD'].apply(clean_text)
Municipios_20['MUNICIPIO_c_benef'] = Municipios_20['MUNICIPIO'].apply(clean_text)

In [14]:
# Creamos las dos keys de beneficiarios 
Municipios_20['KEY_benef_mun'] = Municipios_20['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_20[
    'MUNICIPIO_c_benef'].astype(str)

In [15]:
Municipios_20.shape

(182, 5)

In [16]:
Municipios_20.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2020

Obtener listado único de municipios y listado único de localidades de inegi de 2020 con sus claves.

In [17]:
path_dataset_inegi_2020 = '../../data/inegi/dataset_inegi_clean_2020.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2020 = pd.read_csv(path_dataset_inegi_2020)

### 1.2.1 INEGI 2020 Municipios únicos para cada año.

In [18]:
dataset_inegi_2020['KEY_inegi_municipio'] = dataset_inegi_2020['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2020[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2020['KEY_inegi_localidad'] = dataset_inegi_2020['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2020[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2020['Localidad_c_inegi'].astype(str)

In [19]:
INEGI_UNIQUEMUN_2020 = dataset_inegi_2020.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2020 = INEGI_UNIQUEMUN_2020.drop_duplicates()

In [20]:
INEGI_UNIQUEMUN_2020.shape

(98356, 8)

In [21]:
INEGI_UNIQUEMUN_2020.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi', 'POB_TOTAL',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

### 1.2.1 INEGI 2020 Localidades únicas para cada año.

In [22]:
INEGI_UNIQUELOC_2020 = dataset_inegi_2020

INEGI_UNIQUELOC_2020 = INEGI_UNIQUELOC_2020.drop_duplicates()

In [23]:
INEGI_UNIQUELOC_2020.shape

(300690, 12)

In [24]:
# Ordenar el DataFrame por Entidad_inegi, Municipio_inegi y Localidad_inegi
df_sorted = INEGI_UNIQUELOC_2020.sort_values(by=['Entidad_inegi', 'Municipio_inegi', 'Localidad_inegi'])

# Identificar duplicados basados en Entidad_inegi, Municipio_inegi y Localidad_inegi
duplicados = df_sorted[df_sorted.duplicated(subset=['Entidad_inegi', 'Municipio_inegi', 'Localidad_inegi'], keep=False)]
print("Número de rows con base en Entidad, Municipio y Localidad: " , duplicados.shape)

Número de rows con base en Entidad, Municipio y Localidad:  (43858, 12)


In [25]:
conteo_duplicados = duplicados.groupby(['Entidad_inegi', 'Municipio_inegi', 'Localidad_inegi']).size().reset_index(name='Numero_Duplicados')

# Ordenar por el número de duplicados en orden descendente
conteo_duplicados = conteo_duplicados.sort_values(by='Numero_Duplicados', ascending=False)

# Mostrar el conteo de duplicados ordenado por count
conteo_duplicados

,Entidad_inegi,Municipio_inegi,Localidad_inegi,Numero_Duplicados
11516,Sinaloa,Culiacán,Ninguno,95
1358,Campeche,Carmen,Ninguno,79
11273,Sinaloa,Ahome,Ninguno,63
12054,Sonora,Cajeme,Ninguno,58
16229,Yucatán,Tizimín,San José,53
...,...,...,...,...
6859,Durango,San Dimas,Cuevecillas,2
6860,Durango,San Dimas,El Carrizo,2
6861,Durango,San Dimas,El Chirimoyo,2
6862,Durango,San Dimas,El Gato,2


In [26]:
conteo_duplicados_2020 = conteo_duplicados[conteo_duplicados['Entidad_inegi'].isin(['Guerrero', 'Morelos', 'Tlaxcala', 'Puebla'])]

In [27]:
conteo_duplicados_2020

,Entidad_inegi,Municipio_inegi,Localidad_inegi,Numero_Duplicados
10680,Puebla,San Salvador Huixcolotla,San Salvador Huixcolotla,6
7643,Guerrero,Zirándaro,El Guayabo,6
7665,Guerrero,Zirándaro,Las Juntas,5
7594,Guerrero,Técpan de Galeana,El Rincón,5
10534,Puebla,Chapulco,Chapulco,5
...,...,...,...,...
7389,Guerrero,Acatepec,Ojo de Agua,2
7385,Guerrero,Acapulco de Juárez,El Paraíso,2
7386,Guerrero,Acapulco de Juárez,El Rincón,2
7387,Guerrero,Acapulco de Juárez,El Zapote,2


# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2020, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_20 y el dataset de INEGI correspondiente.

In [24]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [25]:
diccionario_MUN_20 = fuzzy_merge_benef2019_2022(Municipios_20, INEGI_UNIQUEMUN_2020, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_20.shape

(182, 14)

In [26]:
diccionario_MUN_20.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [27]:
diccionario_MUN_20.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_20.csv', index=False)

In [28]:
diccionario_MUN_20_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_20_simple.csv')

In [29]:
diccionario_MUN_20_simple.shape

(183, 2)

In [30]:
diccionario_MUN_20_simple.columns

Index(['KEY_benef_mun', 'KEY_inegi_municipio'], dtype='object')

In [31]:
nan_rows = beneficiarios_20[beneficiarios_20.isna().any(axis=1)]

In [32]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [33]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_20[beneficiarios_20['ENTIDAD'] == 'NACIONAL']

In [34]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [35]:
beneficiarios_20.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [36]:
beneficiarios_20.shape

(369037, 16)

In [37]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_20['ESTADO_Clean'] = beneficiarios_20['ENTIDAD'].apply(clean_text)
beneficiarios_20['MUNICIPIO_Clean'] = beneficiarios_20['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_20['Estado-mun-KEY'] = beneficiarios_20['ESTADO_Clean'].astype(str) + '-' + beneficiarios_20[
    'MUNICIPIO_Clean'].astype(str)

In [38]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_20, diccionario_MUN_20_simple, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef_mun", how='left')

In [39]:
listado_beneficiarios_parte_I.shape

(369037, 21)

In [40]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [41]:
listado_beneficiarios_parte_I.isna().sum()

BENEFICIARIO           0
ZONA                   0
ENTIDAD                0
MUNICIPIO              0
LOCALIDAD              0
ESTRATIFICACIÓN        0
PROGRAMA               0
COMPONENTE             0
SUBCOMPONENTE          0
PRODUCTO               0
FECHA                  0
MONTO FEDERAL          0
APOYO                  0
ACTIVIDAD              0
ESLABÓN                0
CICLO AGRÍCOLA         0
ESTADO_Clean           0
MUNICIPIO_Clean        0
Estado-mun-KEY         0
KEY_benef_mun          0
KEY_inegi_municipio    0
dtype: int64

In [42]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_I[listado_beneficiarios_parte_I.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [43]:
rows_with_nan

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA,ESTADO_Clean,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio


In [44]:
# Hacer el join de la Parte II
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2020, left_on="KEY_inegi_municipio",
                                        right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [45]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,...,MUNICIPIO_Clean,Estado-mun-KEY,KEY_benef_mun,KEY_inegi_municipio,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,Entidad_c_inegi,Municipio_c_inegi
0,ESPINOZA ZAMORA SILVANO,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
1,CASTRO IGNACIO ESTEBAN,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
2,ESPINOZA ESPINOZA AMELIA,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
3,DIRCIO CALIXTO ANUNCIACION,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
4,CALIXTRO LOPEZ LAURA,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Muy alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acatepec,guerrero-acatepec,guerrero-acatepec,guerrero-acatepec,12,Guerrero,76,Acatepec,guerrero,acatepec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369032,RAMIREZ FLORES GUILLERMINA,CENTRO PAÍS,PUEBLA,ACAJETE,LA MAGDALENA TETELA MORELOS,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,acajete,puebla-acajete,puebla-acajete,puebla-acajete,21,Puebla,1,Acajete,puebla,acajete
369033,MARTINEZ HERNANDEZ JOSE JUAN,CENTRO PAÍS,PUEBLA,SAN SALVADOR EL SECO,SAN SALVADOR EL SECO,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,san salvador el seco,puebla-san salvador el seco,puebla-san salvador el seco,puebla-san salvador el seco,21,Puebla,142,San Salvador el Seco,puebla,san salvador el seco
369034,VARILLAS MORA JUANA,CENTRO PAÍS,PUEBLA,TLACOTEPEC DE BENITO JUÁREZ,TLACOTEPEC DE BENITO JUÁREZ,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,tlacotepec de benito juarez,puebla-tlacotepec de benito juarez,puebla-tlacotepec de benito juarez,puebla-tlacotepec de benito juarez,21,Puebla,177,Tlacotepec de Benito Juárez,puebla,tlacotepec de benito juarez
369035,JIMENEZ AGUILAR FLORENTINO,CENTRO PAÍS,GUERRERO,JUCHITÁN,JUCHITÁN,Alto,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,...,juchitan,guerrero-juchitan,guerrero-juchitan,guerrero-juchitan,12,Guerrero,80,Juchitán,guerrero,juchitan


In [46]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi'],
      dtype='object')

In [47]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio', 'Entidad_c_inegi','Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD', 'ESLABÓN'])

In [48]:
listado_beneficiarios_parte_II

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
0,ESPINOZA ZAMORA SILVANO,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Cultivo de maíz grano,2020-06-12,6163.44,PV2020,guerrero-acatepec,12,Guerrero,76,Acatepec
1,CASTRO IGNACIO ESTEBAN,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Cultivo de maíz grano,2020-06-14,6163.44,PV2020,guerrero-acatepec,12,Guerrero,76,Acatepec
2,ESPINOZA ESPINOZA AMELIA,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Cultivo de maíz grano,2020-06-15,3081.72,PV2020,guerrero-acatepec,12,Guerrero,76,Acatepec
3,DIRCIO CALIXTO ANUNCIACION,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Cultivo de maíz grano,2020-06-15,3081.72,PV2020,guerrero-acatepec,12,Guerrero,76,Acatepec
4,CALIXTRO LOPEZ LAURA,CENTRO PAÍS,GUERRERO,ACATEPEC,ESCALERILLA ZAPATA,Cultivo de maíz grano,2020-06-15,3081.72,PV2020,guerrero-acatepec,12,Guerrero,76,Acatepec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369032,RAMIREZ FLORES GUILLERMINA,CENTRO PAÍS,PUEBLA,ACAJETE,LA MAGDALENA TETELA MORELOS,Cultivo de maíz grano,2021-03-31,2816.79,PV2020,puebla-acajete,21,Puebla,1,Acajete
369033,MARTINEZ HERNANDEZ JOSE JUAN,CENTRO PAÍS,PUEBLA,SAN SALVADOR EL SECO,SAN SALVADOR EL SECO,Cultivo de maíz grano,2021-03-31,5633.58,PV2020,puebla-san salvador el seco,21,Puebla,142,San Salvador el Seco
369034,VARILLAS MORA JUANA,CENTRO PAÍS,PUEBLA,TLACOTEPEC DE BENITO JUÁREZ,TLACOTEPEC DE BENITO JUÁREZ,Cultivo de maíz grano,2021-03-31,2816.79,PV2020,puebla-tlacotepec de benito juarez,21,Puebla,177,Tlacotepec de Benito Juárez
369035,JIMENEZ AGUILAR FLORENTINO,CENTRO PAÍS,GUERRERO,JUCHITÁN,JUCHITÁN,Cultivo de maíz grano,2020-05-28,3082.00,PV2020,guerrero-juchitan,12,Guerrero,80,Juchitán


In [49]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [50]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2020.csv', index=False)

In [51]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO       0
ZONA               0
ENTIDAD            0
MUNICIPIO          0
LOCALIDAD          0
PRODUCTO           0
FECHA              0
MONTO FEDERAL      0
CICLO AGRÍCOLA     0
KEY_benef_mun      0
CVE_ENT            0
Entidad_inegi      0
CVE_MUN            0
Municipio_inegi    0
dtype: int64

In [52]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II[listado_beneficiarios_parte_II.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [53]:
rows_with_nan

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
